In [106]:
import pandas as pd
import numpy as np
import re

In [107]:
df = pd.read_table("checkins.dat", sep="|")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [108]:
df = df.drop(0, axis = 0)

In [109]:
cols = [x.strip() for x in df.columns.to_list()]
df.columns = cols

In [110]:
for i in ['latitude', 'longitude']:
    df[i][df[i].apply(lambda i: True if re.search('^\s*$', str(i)) else False)]=None

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [111]:
df = df.dropna(subset = ['latitude', 'longitude'], axis = 0)

In [112]:
df = df.reset_index(drop= True)

In [113]:
df.shape

(396634, 6)

In [114]:
from sklearn.cluster import MeanShift

In [115]:
df.dtypes

id             object
user_id       float64
venue_id      float64
latitude       object
longitude      object
created_at     object
dtype: object

In [116]:
for i in df.columns:
    if df[i].dtype == np.dtype('object'):
        df[i] = df[i].str.strip()

In [117]:
df.columns[df.columns == np.object()]

Index([], dtype='object')

In [118]:
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
1,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
2,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
3,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18


In [119]:
for i in ['latitude', 'longitude']:
    df[i] = pd.to_numeric(df[i], downcast="float")

In [120]:
clustering = MeanShift(bandwidth=0.1).fit(df[['latitude', 'longitude']].iloc[0:100000])

In [121]:
clustering.get_params

<bound method BaseEstimator.get_params of MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=None, seeds=None)>

In [122]:
preds = clustering.predict(df[['latitude', 'longitude']].iloc[0:100000])

In [123]:
(unique, counts) = np.unique(preds, return_counts=True)
frequencies = np.asarray((unique, counts)).T

In [124]:
frequencies = frequencies[frequencies[:,1] > 15]

In [126]:
centers = []

for i in frequencies[:,0]:
    indexes = np.where(preds == i)
    lat_mean = np.mean(df['latitude'].iloc[indexes])
    long_mean = np.mean(df['longitude'].iloc[indexes])
    centers.append([lat_mean, long_mean])

In [127]:
offices = [[33.751277, -118.188740] , [25.867736, -80.324116], [51.503016, -0.075479],
           [52.378894, 4.885084], [39.366487, 117.036146], [-33.868457, 151.205134]]

In [144]:
dists = []

for office in offices:
    dist_cluster = []
    for cluster in centers:
        dist_cluster.append(np.linalg.norm(np.array(office) - np.array(cluster)))
    dists.append(dist_cluster)
dists = np.array(dists)    

In [146]:
dists.shape

(6, 591)

In [143]:
dists[0]

array([44.7437248 ,  6.16675359,  6.33702625, 31.62279337,  5.79708327,
       41.46082955, 33.79166424, 47.89041221, 14.4609169 ,  3.83513602,
        0.30558591, 27.33679913, 20.74101241, 44.93153049, 14.49644589,
       43.46966248,  0.38259155,  1.35221419, 21.43954137,  5.18359898])

In [147]:
np.where(dists == np.amin(dists))

(array([5], dtype=int64), array([416], dtype=int64))

In [149]:
f = open('answer.txt', 'w+')
f.write(f'{centers[416][0]} {centers[416][1]}')
f.close()